<a href="https://colab.research.google.com/github/quanvu20702/Trafic-Sign-Classification/blob/main/Traffic_Sign_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Traffic Sign Classification use dataset German Traffic Sign Recognition Benchmark (GTSRB)
Link dataset: https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip

###Install library

In [1]:
import os
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import keras
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential

###Dowload dataset

In [2]:
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip

--2023-04-20 15:02:17--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276294756 (263M) [application/zip]
Saving to: ‘GTSRB_Final_Training_Images.zip’

GTSRB_Final_Trainin 100%[===================>] 263.50M  24.6MB/s    in 11s     

2023-04-20 15:02:29 (22.9 MB/s) - ‘GTSRB_Final_Training_Images.zip’ saved [276294756/276294756]



In [3]:
!unzip GTSRB_Final_Training_Images.zip

Streaming output truncated to the last 5000 lines.
  inflating: GTSRB/Final_Training/Images/00035/00000_00019.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00020.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00021.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00022.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00023.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00024.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00025.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00026.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00027.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00028.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00029.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00000.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00001.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00002.ppm  
  inflating: GTSRB/Final_Training/Image

###Read data

In [4]:
def load_data(input_size = (64,64), data_path = '/content/GTSRB/Final_Training/Images'):

  pixels = []
  labels = []

  for dir in os.listdir(data_path):
    if dir == '.DS_Store':
      continue
    
    class_dir = os.path.join(data_path, dir)
    info_file = pd.read_csv(os.path.join(class_dir, "GT-" + dir + '.csv'), sep=';')

    for row in info_file.iterrows():
      pixel = imread(os.path.join(class_dir, row[1].Filename))
      pixel = pixel[row[1]['Roi.X1']:row[1]['Roi.X2'], row[1]['Roi.Y1']:row[1]['Roi.Y2'], :]
      img = cv2.resize(pixel, input_size)

      pixels.append(img)
      labels.append(row[1].ClassId)
  return pixels, labels

###Image path

In [5]:
data_path = '/content/GTSRB/Final_Training/Images'
pixels, labels = load_data(data_path=data_path)

<ipython-input-4-144add2cbf3c>:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pixel = imread(os.path.join(class_dir, row[1].Filename))
<ipython-input-4-144add2cbf3c>:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pixel = imread(os.path.join(class_dir, row[1].Filename))
<ipython-input-4-144add2cbf3c>:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pixel = imread(os.path.join(

###Split data train, val, test

In [6]:
def split_train_val_test_data(pixels, labels):

  pixels = np.array(pixels)
  labels = keras.utils.np_utils.to_categorical(labels)

  randomize = np.arange(len(pixels))
  np.random.shuffle(randomize)
  X = pixels[randomize]
  print("X = ", X.shape)
  y = labels[randomize]

  train_size = int(X.shape[0] * 0.6)
  X_train, X_val = X[:train_size], X[train_size:]
  y_train, y_val = y[:train_size], y[train_size:]

  val_size = int(X_val.shape[0] * 0.5)
  X_val, X_test = X_val[:val_size], X_val[val_size:]
  y_val, y_test = y_val[:val_size], y_val[val_size:]

  return X_train, y_train, X_val, y_val, X_test, y_test
  
X_train, y_train, X_val, y_val, X_test, y_test = split_train_val_test_data(pixels, labels)

X =  (39209, 64, 64, 3)


###Build model

In [7]:
def build_model(input_shape=(64,64,3), filter_size = (3,3), pool_size = (2, 2), output_size = 43):
    model = Sequential([
        Conv2D(16, filter_size, activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        Conv2D(16, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Conv2D(64, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Flatten(),
        Dense(2048, activation='relu'),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(output_size, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
    model.summary()
    return model
model = build_model(input_shape=(64,64,3), output_size=43)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 16)       64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 16)        2320      
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 16)       64        
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                        

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


###Train model

In [8]:
epochs = 10
batch_size = 16

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                               validation_data=(X_val, y_val))

model.save("traffic_sign_model.h5")

Epoch 1/10
1471/1471 [==============================] - 28s 12ms/step - loss: 2.4544 - accuracy: 0.3476 - val_loss: 0.8695 - val_accuracy: 0.7497
Epoch 2/10
1471/1471 [==============================] - 16s 11ms/step - loss: 0.7913 - accuracy: 0.7626 - val_loss: 0.2618 - val_accuracy: 0.9203
Epoch 3/10
1471/1471 [==============================] - 15s 10ms/step - loss: 0.3395 - accuracy: 0.8948 - val_loss: 0.1209 - val_accuracy: 0.9660
Epoch 4/10
1471/1471 [==============================] - 16s 11ms/step - loss: 0.1852 - accuracy: 0.9433 - val_loss: 0.0811 - val_accuracy: 0.9750
Epoch 5/10
1471/1471 [==============================] - 16s 11ms/step - loss: 0.1188 - accuracy: 0.9619 - val_loss: 0.0601 - val_accuracy: 0.9816
Epoch 6/10
1471/1471 [==============================] - 15s 10ms/step - loss: 0.0849 - accuracy: 0.9741 - val_loss: 0.0432 - val_accuracy: 0.9866
Epoch 7/10
1471/1471 [==============================] - 16s 11ms/step - loss: 0.0631 - accuracy: 0.9801 - val_loss: 0.0374 -

###Test

In [9]:
model.evaluate(X_test, y_test)

246/246 [==============================] - 1s 4ms/step - loss: 0.0267 - accuracy: 0.9922


[0.026679806411266327, 0.9922213554382324]